In [1]:
from scipy import optimize
from bingo.evaluation.fitness_function import VectorBasedFunction
import numpy as np
import sys
from numpy import array

In [2]:
number_of_eqps_values = 21
eqps_values_np = np.array([ i*0.0015 for i in range(number_of_eqps_values) ])
P_desired = np.array([[1, -0.5, -0.5],
                    [-0.5, 1, -0.5],
                    [-0.5, -0.5, 1]])
dataloc = 'processed_data/vm_1_bingo_format.txt'
pre_data = np.loadtxt(dataloc)
print(pre_data)
training_data = []
eqpses = []
for row in pre_data:
    if np.isnan(row[0]) == False:
        training_data.append(row[:3])
        eqpses.append(row[3])
training_data = np.array(training_data)
eqpses = np.array(eqpses)
#print(eqpses)
print(training_data)

def mapping_(x):

    # mapping = lambda X_0: array([[x[0], x[1], x[2]],
    #    [x[3], x[4], x[5]],
    #    [x[6], x[7], x[8]]]) + (X_0) * (array([[x[9], x[10], x[11]],
    #    [x[12], x[12], x[13]],
    #    [x[14], x[15], x[16]]]))
    # mapping = lambda X_0: (x[1]*X_0 + x[0])  * (array([[x[2], x[3], x[4] ],  
    #     [x[3] , x[6], x[5]],
    #     [x[4], x[5], x[7]]]))#*(10 + 990*X_0)
    # mapping = lambda X_0: (x[1]*X_0 + x[0])  * (array([[x[2], 0, 0 ],  
    #     [0 , x[6], 0],
    #     [0, 0, x[7]]]))
    # ((array(0.00500253) + array(0.00500253))/(X_0 + array(0.00500253) + array(0.00500253)) ) * (array([[-0.87240378, -1.01069406, -0.3899734 ],
    #    [-0.8733965 , -1.02045703, -0.37967495],
    #    [-0.86253049, -1.02066836, -0.38872487]]))
    mapping = lambda X_0: ((x[0])/(X_0 + x[1]) ) * (array([[x[2], x[3], x[4] ],  
        [x[3] , x[6], x[5]],
        [x[4], x[5], x[7]]]))
    
    mapping_matrices = np.array([mapping(e) for e in eqpses])
    P_mapped = np.transpose(mapping_matrices,(0, 2, 1)) @ P_desired @ mapping_matrices
    back_calculated_yield_stresses = []
    for i,stresses in enumerate(training_data):
        #print('here we go', np.transpose(training_data[i]), P_mapped[i], training_data[i])
        back_calculated_yield_stresses.append(np.transpose(training_data[i]) @ P_mapped[i] @ training_data[i])
        #back_calculated_yield_stresses.append(P_mapped[i] @ training_data[i])
        #print(P_mapped[i] @ training_data[i])
    #print(back_calculated_yield_stresses)

    error = np.ones(len(training_data)) - np.array(back_calculated_yield_stresses)
    #error = training_data - np.array(back_calculated_yield_stresses)
    #print(error)
    return error.flatten()



# ((array(0.00500253) + array(0.00500253))/(X_0 + array(0.00500253) + array(0.00500253)) ) * (array([[-0.87240378, -1.01069406, -0.3899734 ],  
#        [-0.8733965 , -1.02045703, -0.37967495],
#        [-0.86253049, -1.02066836, -0.38872487]]))


[[ 2.11324865e+00 -7.88675135e+00  5.77350269e+00  0.00000000e+00]
 [ 4.20536482e+00 -1.56946352e+01  1.14892704e+01  1.00000000e-02]
 [ 6.29748099e+00 -2.35025190e+01  1.72050380e+01  2.00000000e-02]
 ...
 [ 3.97713397e+01 -1.48428660e+02  1.08657321e+02  1.80000000e-01]
 [ 4.18634558e+01 -1.56236544e+02  1.14373088e+02  1.90000000e-01]
 [ 4.39555720e+01 -1.64044428e+02  1.20088856e+02  2.00000000e-01]]
[[   2.11324865   -7.88675135    5.77350269]
 [   4.20536482  -15.69463518   11.48927036]
 [   6.29748099  -23.50251901   17.20503802]
 ...
 [  39.77133967 -148.42866033  108.65732066]
 [  41.86345584 -156.23654416  114.37308833]
 [  43.955572   -164.044428    120.08885599]]


In [3]:
x0 = [-0.83111601, -0.46657978, -0.06323608, -0.83507571, -0.45586107, -0.0702321, -0.84231627, -0.46026723, -0.05982535, 
      67.83825554, 11.92019618, 22.12241255, 67.75485734, 11.34285823, 22.7518725, 68.42081951, 11.30870116, 22.22108227
]
x0 = [ 0.00500253*2,0.00500253*2,-0.87240378, -1.01069406, -0.3899734, -0.8733965 , -1.02045703, -0.37967495 ]

mapping_(x0)
#results = optimize.minimize(mapping_, )

array([-8.02612469, -7.94061018, -7.91218622, ..., -7.86440578,
       -7.86395838, -7.8635536 ])

In [4]:
results = optimize.root(mapping_, x0=x0, method='lm', options={"xtol": 1e-16, "gtol": 1e-16, "maxiter": 100000})
results

   cov_x: array([[ 2.69312105e+05, -4.17704282e-02, -5.25404090e+05,
        -6.14672092e+05, -5.59787642e+05, -6.36789016e+05,
        -1.11693538e+06, -1.68909046e+05],
       [-4.17494891e-02,  4.49490503e-07, -7.64194654e-02,
        -1.87163547e-01,  1.38760735e-02,  1.04800438e-01,
        -5.72672709e-02,  6.51997524e-02],
       [-5.24961837e+05, -7.59571278e-02,  4.40121525e+07,
        -5.29811066e+06,  1.24161461e+07,  1.19771358e+07,
        -1.13821738e+07, -1.35348029e+07],
       [-6.15790614e+05, -1.86363459e-01, -5.24095575e+06,
         5.37629049e+07,  2.51564628e+06,  1.81007700e+07,
         7.60655473e+07,  3.55470305e+06],
       [-5.60064197e+05,  1.41278829e-02,  1.24418546e+07,
         2.48561796e+06,  6.91574808e+06,  1.80704672e+06,
        -2.38392419e+06,  1.15048343e+06],
       [-6.33681219e+05,  1.05868964e-01,  1.19522060e+07,
         1.79111756e+07,  1.77977206e+06,  4.25563233e+07,
         5.50744556e+07, -4.77940790e+06],
       [-1.11532431e+06,

In [5]:
np.set_printoptions(threshold=sys.maxsize)
mapping_(results.x)

array([-1.97619698e-14,  3.16413562e-14, -2.64233080e-14,  3.88578059e-15,
       -3.28626015e-14,  3.23074900e-14,  6.99440506e-15,  1.31006317e-14,
       -3.10862447e-15,  8.65973959e-15, -5.32907052e-15, -1.79856130e-14,
       -1.04360964e-14,  1.99840144e-14,  2.38697950e-14, -2.22044605e-16,
       -7.77156117e-15, -1.77635684e-14, -1.55431223e-14, -1.59872116e-14,
       -7.32747196e-15, -2.59792188e-14,  2.35367281e-14, -2.75335310e-14,
        6.66133815e-15, -1.79856130e-14,  3.31956684e-14,  6.21724894e-15,
        7.10542736e-15,  2.22044605e-16,  1.24344979e-14, -7.32747196e-15,
       -2.10942375e-14, -3.77475828e-15,  3.06421555e-14,  2.18713936e-14,
        4.10782519e-15, -1.17683641e-14, -7.99360578e-15, -1.28785871e-14,
       -2.26485497e-14, -6.88338275e-15, -1.33226763e-14,  1.94289029e-14,
       -1.82076576e-14,  5.66213743e-15,  2.22044605e-16,  3.25295346e-14,
        1.24344979e-14,  1.33226763e-14, -3.55271368e-15,  2.16493490e-14,
       -2.44249065e-15, -

In [6]:
np.set_printoptions(precision=32)
results.x

array([ 0.0314091528502507 ,  0.01010101010101011, -0.3926047752324647 ,
       -0.7336831417939027 , -0.2434946890006791 , -0.6050921964480506 ,
       -1.0431560121827226 , -0.14650923982744513])

In [7]:
# mapping = lambda X_0: array([[-0.83111601, -0.46657978, -0.06323608],
#        [-0.83507571, -0.45586107, -0.07023218],
#        [-0.84231627, -0.46026723, -0.05982535]]) + (X_0) * (array([[67.83825554, 11.92019618, 22.12241255],
#        [67.75485734, 11.34285823, 22.75187253],
#        [68.42081951, 11.30870116, 22.22108227]]))
# Q = mapping(0)
# Q